In [458]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime
from bs4 import BeautifulSoup
import requests

from pyomo.environ import AbstractModel, ConcreteModel, SolverFactory
from pyomo.environ import Set, Constraint, Objective, Param, Var
from pyomo.environ import Binary
from pyomo.environ import minimize, maximize
from pyomo.environ import value

import warnings
warnings.filterwarnings('ignore')


# Load/Clean Data

In [459]:

def load_data(filepath):
    df = pd.read_excel(filepath, engine='openpyxl')
    return df


def clean_data(data):
    # Drop columns with null values
    data_cleaned = data.dropna(inplace=False)

    #clean up Oz column 
    data_cleaned.rename(columns = {'#Oz':'Oz'}, inplace = True)
    data_cleaned['Oz'] = data_cleaned['Oz'].replace(regex=[r'\D+'], value="")
    data_cleaned['Oz'] = round(data_cleaned['Oz'].astype(float), 2)
    
    # Clean number of shipments column
    data_cleaned['Number of Shipments'] = pd.to_numeric(data_cleaned['Number of Shipments'], errors = 'coerce')
    data_cleaned.dropna(inplace=True)
    data_cleaned['Number of Shipments'] = data_cleaned['Number of Shipments'].astype(int)
    
    # Add pounds column
    data_cleaned['Pounds'] = round(data_cleaned['Oz'].astype(float) / 16, 2)
    
    # Determine average number of Pounds/Oz per shipment
    data_cleaned['Pounds_per_Shipment'] = round(data_cleaned['Pounds'] / data_cleaned['Number of Shipments'], 2)
    data_cleaned['Oz_per_Shipment'] = round(data_cleaned['Oz'] / data_cleaned['Number of Shipments'], 2)
    
    # Create a feature capturing the number of months since the last donation
    data_cleaned['DOLD'] = data_cleaned['DOLD'].replace('10/0/3/2019', '10/3/2019')
    data_cleaned['Current_Date'] = pd.to_datetime(date.today())
    data_cleaned['Months_since_last_donation'] = ((pd.to_datetime(data_cleaned['Current_Date']).dt.year - pd.to_datetime(data_cleaned['DOLD']).dt.year) * 12) + (pd.to_datetime(data_cleaned['Current_Date']).dt.month - pd.to_datetime(data_cleaned['DOLD']).dt.month)
    
    # strip the origin column
    data_cleaned['Origin'] = data_cleaned['Origin'].str.strip()  
    
    # Add column to track if it was dropped off or not
    data_cleaned['droppedOff'] = data_cleaned['Origin'] != 'Shipped from Donor'
    
    # Return the cleaned data
    return data_cleaned


def generate_zip_data(data):
    # Get unique zipcodes
    zipcode_list = data['Zip code'].unique()

    # Scrape the coordinates
    latitude = []
    longitude = []
    
    for zip_code in zipcode_list:
        url = 'https://www.zipdatamaps.com/{}'.format(zip_code)
        html=requests.get(url)
        Soup =BeautifulSoup(html.content , 'html.parser')
        table = Soup.find(attrs={'class': "table table-striped table-bordered table-hover table-condensed"})
        
        data = [] 
        for i in table.find_all('tr'):
            data.append([j.text for j in i.find_all('td')])
        coord = data[-1][1:]
        
        lat, long = coord[0].split(',')
        latitude.append(lat)
        longitude.append(long)

    # Create the dataframe
    zipcode_df = pd.DataFrame({
        "Zip code": zipcode_list,
        "Latitude": latitude,
        "Longitude":longitude
    })
    
        
    # Return the zipcode dataframe
    return zipcode_df

In [460]:
def calculate_shipping_cost_pounds(weight_in_pounds=0):
    if(weight_in_pounds <= 2):
        return 41
    elif(weight_in_pounds <= 3):
        return 45.76
    elif(weight_in_pounds <= 4):
        return 49.92
    elif(weight_in_pounds <= 5):
        return 50.34 
    elif(weight_in_pounds <= 6):
        return 57.70
    elif(weight_in_pounds <= 7):
        return 58.44
    elif(weight_in_pounds <= 8):
        return 58.70
    else:
        return 59.49
    
def calculate_shipping_cost_ounces(weight_in_ounces):
    # Convert weight to pounds 
    weight_in_pounds = round(weight_in_ounces / 16, 2)
    return calculate_shipping_cost_pounds(weight_in_pounds)
    

In [461]:
path_to_data = os.path.join(os.getcwd(), 'NYMB_updates.xlsx')

# Load and clean the data
data = load_data(path_to_data)
data_cleaned = clean_data(data)

# Get coordinates for zipcodes
zipcode_df = generate_zip_data(data)

# Merge the datasets
data_cleaned = pd.merge(data_cleaned, zipcode_df, how='inner', on='Zip code')

# Reorganize the columns
data_cleaned = data_cleaned[['Donor Number', 'DOFD', 'DOLD', 'Months_since_last_donation', 'Origin', 'Neighborhood', 'Zip code', 'Number of Shipments', 'Oz', 'Oz_per_Shipment', 'droppedOff']]

display(data_cleaned)


,Donor Number,DOFD,DOLD,Months_since_last_donation,Origin,Neighborhood,Zip code,Number of Shipments,Oz,Oz_per_Shipment,droppedOff
0,1,2022-08-02,2022-08-02 00:00:00,3,Shipped from Donor,Battery Park,10280,1,247.00,247.00,False
1,5,2021-04-09,2021-05-07 00:00:00,18,Shipped from Donor,Battery Park,10280,2,603.00,301.50,False
2,6,2022-02-24,2022-02-24 00:00:00,9,Shipped from Donor,Battery Park,10280,1,493.84,493.84,False
3,2,2019-04-10,2019-05-10 00:00:00,42,Columbia Midtown,Battery Park,10282,3,596.00,198.67,True
4,3,2019-05-17,2019-05-17 00:00:00,42,Shipped from Donor,Battery Park,10282,1,360.00,360.00,False
...,...,...,...,...,...,...,...,...,...,...,...
249,256,2022-06-02,2022-06-02 00:00:00,5,Shipped from Donor,Washington Heights (upper west side),10040,2,342.00,171.00,False
250,257,2020-09-10,2020-09-10 00:00:00,26,Shipped from Donor,Washington Heights (upper west side),10040,1,208.00,208.00,False
251,258,2022-01-12,2022-02-11 00:00:00,9,Shipped from Donor,Washington Heights (upper west side),10040,3,847.00,282.33,False
252,259,2022-03-01,2022-08-04 00:00:00,3,Shipped from Donor,Washington Heights (upper west side),10040,4,1104.00,276.00,False


In [462]:
# Remove anyone who hasn't donated in the past three years
data_cleaned = data_cleaned[data_cleaned['Months_since_last_donation'] <= 24]
display(data_cleaned)

,Donor Number,DOFD,DOLD,Months_since_last_donation,Origin,Neighborhood,Zip code,Number of Shipments,Oz,Oz_per_Shipment,droppedOff
0,1,2022-08-02,2022-08-02 00:00:00,3,Shipped from Donor,Battery Park,10280,1,247.00,247.00,False
1,5,2021-04-09,2021-05-07 00:00:00,18,Shipped from Donor,Battery Park,10280,2,603.00,301.50,False
2,6,2022-02-24,2022-02-24 00:00:00,9,Shipped from Donor,Battery Park,10280,1,493.84,493.84,False
7,9,2022-06-01,2022-06-23 00:00:00,5,Shipped from Donor,Chelsea,10001,2,381.00,190.50,False
8,10,2021-03-03,2021-03-03 00:00:00,20,Shipped from Donor,Chelsea,10001,1,21.00,21.00,False
...,...,...,...,...,...,...,...,...,...,...,...
244,263,2021-11-17,2022-05-25 00:00:00,6,Shipped from Donor,Washington Heights (upper west side),10033,2,236.34,118.17,False
245,252,2022-04-14,2022-06-21 00:00:00,5,Shipped from Donor,Washington Heights (upper west side),10039,3,607.00,202.33,False
249,256,2022-06-02,2022-06-02 00:00:00,5,Shipped from Donor,Washington Heights (upper west side),10040,2,342.00,171.00,False
251,258,2022-01-12,2022-02-11 00:00:00,9,Shipped from Donor,Washington Heights (upper west side),10040,3,847.00,282.33,False


In [463]:
print(data_cleaned['Origin'].unique())

['Shipped from Donor' 'Directly at NYMB' 'Philippa Gordon, MD']


In [464]:
print(data_cleaned['Neighborhood'].unique())


['Battery Park' 'Chelsea' 'City Hall' 'East Village'
 'Hamilton Heights (upper west side)' 'Harlem' 'Inwood' 'Lower East Side'
 'Midtown-East side' 'Midtown-West side' 'Morningside Heights'
 'Murray Hill' 'NOHO' 'Roosevelt Island' 'Stuyvesant Park' 'Tribeca'
 'upper east side' 'upper west side'
 'Washington Heights (upper west side)']


# Formulation:

### Important Notes:
- Fixed startup = $500
- Cost of shipment in Manhattan is the same, entirely dependent on weight of the shipment. **Consolidation is Key**. *Want to Attack places with the most donations ounces*
    - .1-2 lbs - $41
    - 2-3 - $45.76
    - 3-4 - $49.92 
    - 4-5 - $50.34 
    - 5-6 - $57.70 
    - 6-7 - $58.44
    - 7-8 - $58.70 
    - 8-9 - $59.49
    ...


### Decision Variables: 
- Binary Decision variable for each county of whether or not to place a milk bank 


### Objective: Minimize Cost

- Creation_Cost = 500 * Number of Depos Opened
- Depo_Shipping_Cost = Depo[n] * shipping_cost(average box price based on milk) * number of boxes needed for each n in neighborhoods
- Home_Shipping_Cost = shipping_cost(average box price from home) * number of shipments usually made - Depo[n]*shipping_cost(average box price based on milk) * number of boxes needed for each n in neighborhoods

- Total_Cost = Creation_Cost + Depo_Shipping_Cost + Home_Shipping_Cost

### Constraints
1. Between 2 - 5 Depos
2. 


### Assumptions we are making
- People will donate roughly the same whether they mail it in or drop it off
- Neighborhood donation rate will remain approximately the same in the future
- Only making one shipment from depo, but that isn't necessarily accurate
- Percentage of people who drop off will remain roughly the same
- Only data from past 2 years is relevant
- Assume shipping FedEx

### What we need to figure out
- Do we want a preferance for opening
- option of hiring a part time driver - should we look. Should we provide an option that doesn't include drivers if every other group is?
- more advanced calculation of how much milk will be dropped off by people
- precise shipping costs
- Do we want neighborhood or zipcode granualarity

# Model

In [478]:
# Data
neighborhoods = sorted(data_cleaned['Neighborhood'].unique())
total_oz_per_neighborhood = data_cleaned.groupby('Neighborhood')['Oz'].sum()

total_shipments_per_neighborhood = data_cleaned[data_cleaned['droppedOff'] == False].groupby('Neighborhood')['Number of Shipments'].sum()


amountDroppedOff = data_cleaned[data_cleaned['droppedOff'] == True].groupby('Neighborhood')[['Oz']].sum().reset_index()
display(amountDroppedOff)
#amountDroppedOff = data_cleaned[['Neighborhood']].merge(amountDroppedOff, on='Neighborhood', how='left').fillna(0).groupby('Neighborhood').mean().reset_index()
#amountShipped = total_oz_per_neighborhood.values - amountDroppedOff['Oz'].values

percentageDroppedOff = amountDroppedOff['Oz'].sum() / total_oz_per_neighborhood.loc[amountDroppedOff['Neighborhood']].sum()
#print(total_oz_per_neighborhood.loc[amountDroppedOff['Neighborhood'].values].sum())
print(percentageDroppedOff)

avg_shipment_neighborhood = total_oz_per_neighborhood / total_shipments_per_neighborhood


,Neighborhood,Oz
0,Chelsea,1443.00
1,Midtown-East side,208.12


0.37382297338601034


In [484]:
m = ConcreteModel()

# Sets
m.NEIGHBORHOODS = Set(initialize=neighborhoods)

# Inputs
m.depo_creation_cost = 500
m.avg_shipment_oz = 300 # From Milk Bank Discussion
m.percentage_dropped_off = percentageDroppedOff

m.min_depos = 2
m.max_depos = 5

#m.amount_shipped = Param(m.NEIGHBORHOODS, initialize=amountShipped)
#m.amount_dropped_off = Param(m.NEIGHBORHOODS, initialize=amountDroppedOff)
m.total_oz_ship_neighborhood = Param(m.NEIGHBORHOODS, initialize=total_oz_per_neighborhood)
m.total_shipments_neighborhood = Param(m.NEIGHBORHOODS, initialize=total_shipments_per_neighborhood)
m.avg_shipment_neighborhood = Param(m.NEIGHBORHOODS, initialize=avg_shipment_neighborhood)

# Decision Variables
m.Depo = Var(m.NEIGHBORHOODS, domain=Binary)

# Constraints
def EnoughDepos_rule(m):
    return sum(m.Depo[z] for z in m.NEIGHBORHOODS) >= m.min_depos
m.EnoughDepos_Constraint = Constraint(rule=EnoughDepos_rule)

def TooManyDepos_rule(m):
    return sum(m.Depo[z] for z in m.NEIGHBORHOODS) <= m.max_depos
m.TooManyDepos_Constraint = Constraint(rule=TooManyDepos_rule)

# Objective 
def objective_function(m): # Minimize Cost 
    # TOTAL COST = $500 for each depo + Shipping cost of depos for neighborhood if depo open there + Shipping cost from home * number of shipments
    # If depo is open, subtract the expeceted number of shipments from there 
    
    # Calculate the cost
    creation_cost = sum(m.Depo[z]*m.depo_creation_cost for z in m.NEIGHBORHOODS)
    depo_shipping_cost = sum(m.Depo[z]*calculate_shipping_cost_ounces(m.percentage_dropped_off * m.total_oz_ship_neighborhood[z] / np.ceil(m.percentage_dropped_off * m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z])) * np.ceil(m.percentage_dropped_off * m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z]) for z in m.NEIGHBORHOODS)
    home_shipping_cost = sum(m.Depo[z]*calculate_shipping_cost_ounces((1 - m.percentage_dropped_off) * m.total_oz_ship_neighborhood[z]) * ((np.ceil(1 - m.percentage_dropped_off) * m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z]) - np.ceil(m.percentage_dropped_off * m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z])) + (1 - m.Depo[z])*calculate_shipping_cost_ounces(m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z]) * np.ceil(m.total_oz_ship_neighborhood[z] / m.avg_shipment_neighborhood[z]) for z in m.NEIGHBORHOODS)
    total_cost = creation_cost + depo_shipping_cost + home_shipping_cost
    return total_cost
m.objective = Objective(rule=objective_function, sense=minimize)


In [485]:
# Instantiate the data
#instance_data = {
#    None: {
#    'ZIPCODES': {None: zipcodes}    
#    }
#}
#print(zipcodes)

#instance = m.create_instance(instance_data)
#instance = m.create_instance()

In [486]:
#print(zipcodes)
#print(total_oz_per_zipcode)
#print(total_shipments_per_zipcode)

In [487]:
# Solve the problem
solver = SolverFactory('glpk')
results = solver.solve(m)

# Print the results
print(results['Solver'])
print(m.objective.expr())


- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.06299018859863281

8644.119999999999


In [488]:
CITY_ZIPCODOE_DICT = data_cleaned.set_index('Zip code').to_dict()['Neighborhood']

for n in m.NEIGHBORHOODS:
    if(value(m.Depo[n] == 1)):
        #print(f'{CITY_ZIPCODOE_DICT[zipcode]}: {zipcode}')
        print(n)
        
        
display(total_oz_per_neighborhood.sort_values(ascending=True))

Lower East Side
Roosevelt Island


Neighborhood
Roosevelt Island                           42.00
Lower East Side                           155.21
East Village                              303.00
Inwood                                    306.88
City Hall                                 564.00
Hamilton Heights (upper west side)        846.00
Stuyvesant Park                           927.38
NOHO                                     1041.00
Midtown-East side                        1213.12
Battery Park                             1343.84
Harlem                                   1530.00
Morningside Heights                      1569.68
Murray Hill                              2185.33
Chelsea                                  3203.73
Tribeca                                  3403.88
Washington Heights (upper west side)     4783.82
Midtown-West side                        5849.23
upper west side                          7101.49
upper east side                         13997.51
Name: Oz, dtype: float64